# Retrieval Augmented Generation

We will use the semantic search to provide the best matching wine based on the review description.

### 1. Check PyTorch Version


As in the previous modules, let's import PyTorch and confirm that have have the latest version of PyTorch. The version should already be 1.13.1 or higher. If not, please run the lab in order to get everything set up.

In [1]:
import torch
print(torch.__version__)

1.13.1


### 2. Retrieve notebook variables

The line below will retrieve your shared variables from the previous notebook.

In [2]:
%store -r

### 3. Install OpenSearch ML Python library

In [3]:
!pip install opensearch-py-ml
!pip install accelerate
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 2.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 23.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 33.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 34.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.0/291.0 kB 2.8 MB/s eta 0:00:0000:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.4
    Uninstalling pandas-1.4.4:
      Successfully uninstalled pandas-1.4.4
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.3
    Uninstalling matplotlib-3.5.3:
      Successfully uninstalled matplotlib-3.5.3


Now we need to restart the kernel by running below cell.

In [ ]:
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
restartkernel()

### 4. Import library



In [1]:
import boto3
import re
import time

### 5. Prepare data

This lab combines semantic search with a generative model to present the retrieved data to the user in a conversational tone. This is a dataset of wine reviews. We'll sample this data set to recomend wines that resemble the user provided description.

### Note
You can download the dataset from various sources. One is Kaggle.
https://www.kaggle.com/datasets/christopheiv/winemagdata130k?select=winemag-data-130k-v2.json

After downloading and copying here, unzip in the working directory

In [2]:
#!unzip -o winemag-data-130k-v2.json.zip

In [3]:
import pandas as pd

df = pd.read_json('winemag-data-130k-v2.json')

df.sample(3)

,points,title,description,taster_name,taster_twitter_handle,price,designation,variety,region_1,region_2,province,country,winery
49635,80,Ca' Momi 2007 Rosso Di California Red (Califor...,"Soft and syrupy, with semisweet flavors of ber...",None,None,10.0,Rosso Di California,Red Blend,California,California Other,California,US,Ca' Momi
89344,86,Geografico Castello Tricerchi 2008 Rosso di M...,"Light and luminous in appearance, this bright ...",None,None,NaN,None,Sangiovese Grosso,Rosso di Montalcino,None,Tuscany,Italy,Geografico Castello Tricerchi
25158,88,Dr. Pauly Bergweiler 2012 Wehlener Klosterberg...,"A veritable kaleidoscope of scents, this off-d...",Anna Lee C. Iijima,None,17.0,Wehlener Klosterberg,Riesling,None,None,Mosel,Germany,Dr. Pauly Bergweiler


In [4]:
df.columns

Index(['points', 'title', 'description', 'taster_name',
       'taster_twitter_handle', 'price', 'designation', 'variety', 'region_1',
       'region_2', 'province', 'country', 'winery'],
      dtype='object')

In [5]:
import json
import pandas as pd

# wm_list = df.to_dict('records')
wm_list = df.sample(300,
                   random_state=37).to_dict('records') # sample to keep lab quick

wm_list[:5]

[{'points': 89,
  'title': 'Calera 2014 Pinot Noir (Central Coast)',
  'description': "Relatively light in the glass, this blend of nine vineyards from up and down the coast shows savory baked plum, cardamom dust, earthy cranberry, bay leaf and smoked meat on the nose. It's medium bodied on the palate and very down-the-middle flavor-wise, with cranberry, oregano and thyme elements.",
  'taster_name': 'Matt Kettmann',
  'taster_twitter_handle': '@mattkettmann',
  'price': 28.0,
  'designation': None,
  'variety': 'Pinot Noir',
  'region_1': 'Central Coast',
  'region_2': 'Central Coast',
  'province': 'California',
  'country': 'US',
  'winery': 'Calera'},
 {'points': 92,
  'title': 'Wallis Family Estate 2008 Little Sister Red (Diamond Mountain District)',
  'description': 'Dry and smooth as a fine old Scotch, with intricately woven blackberry, black currant, green olive and herb flavors leading to a long, spicy finish. The tannins are significant, but very finely ground. Classy and pow

### 6. Create an OpenSearch cluster connection.
Next, we'll use Python API to set up connection with OpenSearch Cluster.

Note: if you're using a region other than us-east-1, please update the region in the code below.

#### Get Cloud Formation stack output variables

We also need to grab some key values from the infrastructure we provisioned using CloudFormation. To do this, we will list the outputs from the stack and store this in "outputs" to be used later.

You can ignore any "PythonDeprecationWarning" warnings.

In [6]:
import boto3

cfn = boto3.client('cloudformation')

def get_cfn_outputs(stackname):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

## Setup variables to use for the rest of the demo
cloudformation_stack_name = "semantic-search"

outputs = get_cfn_outputs(cloudformation_stack_name)

bucket = outputs['s3BucketTraining']
aos_host = outputs['OpenSearchDomainEndpoint']

outputs

{'OpenSourceDomainArn': 'arn:aws:es:us-east-1:935924611070:domain/opensearchservi-d3f0c7mvypbf',
 'OpenSearchDomainEndpoint': 'search-opensearchservi-d3f0c7mvypbf-qki7nvr5tkt2bngllgdii2eh2e.us-east-1.es.amazonaws.com',
 'S3BucketSecureURL': 'https://semantic-search-s3buckethosting-nv48itz8gn94.s3.amazonaws.com',
 'SageMakerNotebookURL': 'https://console.aws.amazon.com/sagemaker/home?region=us-east-1#/notebook-instances/openNotebook/NotebookInstance-jcHRY2YDGdBn?view=classic',
 's3BucketTraining': 'semantic-search-s3buckettraining-ht47aafjizwe',
 'Region': 'us-east-1',
 'OpenSearchDomainName': 'opensearchservi-d3f0c7mvypbf',
 's3BucketHostingBucketName': 'semantic-search-s3buckethosting-nv48itz8gn94'}

In [7]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
import boto3

region = 'us-east-1' 

#credentials = boto3.Session().get_credentials()
#auth = AWSV4SignerAuth(credentials, region)
auth = ("master","Semantic123!")
index_name = 'nlp_wmd'

aos_client = OpenSearch(
    hosts = [{'host': aos_host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)

### 7. Configure OpenSearch domain to enable run Machine Learning code in data node

In [8]:
s = b'{"transient":{"plugins.ml_commons.only_run_on_ml_node": false}}'
aos_client.cluster.put_settings(body=s)

{'acknowledged': True,
 'persistent': {},
 'transient': {'plugins': {'ml_commons': {'only_run_on_ml_node': 'false'}}}}

Verify `plugins.ml_commons.only_run_on_ml_node` is set to false

In [9]:
aos_client.cluster.get_settings(flat_settings=True)

{'persistent': {'cluster.routing.allocation.awareness.force.zone.values': 'xx-xxxxx-xx',
  'cluster.routing.allocation.disk.watermark.flood_stage': '0.9758094787597656gb',
  'cluster.routing.allocation.disk.watermark.high': '1.9516189575195313gb',
  'cluster.routing.allocation.disk.watermark.low': '2.9274284362792966gb',
  'cluster.routing.allocation.load_awareness.provisioned_capacity': '1',
  'cluster.routing.allocation.load_awareness.skew_factor': '50.0',
  'cluster_manager.throttling.thresholds.auto-create.value': '200',
  'cluster_manager.throttling.thresholds.cluster-reroute-api.value': '50',
  'cluster_manager.throttling.thresholds.cluster-update-settings.value': '50',
  'cluster_manager.throttling.thresholds.create-component-template.value': '50',
  'cluster_manager.throttling.thresholds.create-data-stream.value': '50',
  'cluster_manager.throttling.thresholds.create-index-template-v2.value': '50',
  'cluster_manager.throttling.thresholds.create-index-template.value': '50',
  '

### 8. Download pre-trained BERT model

In [10]:
import urllib.request
urllib.request.urlretrieve('https://github.com/opensearch-project/ml-commons/raw/2.x/ml-algorithms/src/test/resources/org/opensearch/ml/engine/algorithms/text_embedding/all-MiniLM-L6-v2_torchscript_sentence-transformer.zip?raw=true', 'model/all-MiniLM-L6-v2_torchscript_sentence-transformer.zip')


('model/all-MiniLM-L6-v2_torchscript_sentence-transformer.zip',
 <http.client.HTTPMessage at 0x7efe5e9b7f40>)

Verify model is downloaded successfully in the `model` folder

In [11]:
!ls -al model

total 81468
drwxrwxr-x 2 ec2-user ec2-user     4096 Apr 29 19:22 .
drwxrwxr-x 9 ec2-user ec2-user     4096 Apr 29 19:20 ..
-rw-rw-r-- 1 ec2-user ec2-user      279 Apr 29 17:16 all-MiniLM-L6-v2_torchscript.json
-rw-rw-r-- 1 ec2-user ec2-user 83408741 Apr 29 19:22 all-MiniLM-L6-v2_torchscript_sentence-transformer.zip


### 9. Upload BERT model to OpenSearch domain

In [12]:
from opensearch_py_ml.ml_models import SentenceTransformerModel
from opensearch_py_ml.ml_commons import MLCommonClient

ml_client = MLCommonClient(aos_client)

In [13]:

model_path = './model/all-MiniLM-L6-v2_torchscript_sentence-transformer.zip'
model_config_path = './model/all-MiniLM-L6-v2_torchscript.json'


model_id=ml_client.upload_model(model_path, model_config_path, isVerbose=True)

print("model id:" + model_id)

ml_client.unload_model(model_id)

Total number of chunks 9
Sha1 value of the model file:  9376c2ebd7c83f99ec2526323786c348d2382e6d86576f750c89ea544d6bbb14
Model meta data was created successfully. Model Id:  Nq53zocBnCSv9UI7wh4r
uploading chunk 1 of 9
Model id: {'status': 'Uploaded'}
uploading chunk 2 of 9
Model id: {'status': 'Uploaded'}
uploading chunk 3 of 9
Model id: {'status': 'Uploaded'}
uploading chunk 4 of 9
Model id: {'status': 'Uploaded'}
uploading chunk 5 of 9
Model id: {'status': 'Uploaded'}
uploading chunk 6 of 9
Model id: {'status': 'Uploaded'}
uploading chunk 7 of 9
Model id: {'status': 'Uploaded'}
uploading chunk 8 of 9
Model id: {'status': 'Uploaded'}
uploading chunk 9 of 9
Model id: {'status': 'Uploaded'}
Model uploaded successfully
model id:Nq53zocBnCSv9UI7wh4r


{}

In [14]:
bert_model_id = model_id

### 10. Load the model for inference.

In [15]:
load_model_output = ml_client.load_model(model_id)

print(load_model_output)
task_id = load_model_output['task_id']

{'task_id': 'N653zocBnCSv9UI73R48', 'status': 'CREATED'}


Get the task detailed information.

In [16]:
task_info = ml_client.get_task_info(task_id)

print(task_info)

{'model_id': 'Nq53zocBnCSv9UI7wh4r', 'task_type': 'LOAD_MODEL', 'function_name': 'TEXT_EMBEDDING', 'state': 'RUNNING', 'worker_node': ['HNnmWWEGQEm7JmYmjlIPFg'], 'create_time': 1682796174428, 'last_update_time': 1682796174705, 'is_async': True}


Get the model detailed information.

In [17]:
model_info = ml_client.get_model_info(model_id)

print(model_info)

{'name': 'all-MiniLM-L6-v2', 'algorithm': 'TEXT_EMBEDDING', 'model_version': '1.1.0', 'model_format': 'TORCH_SCRIPT', 'model_state': 'LOADING', 'model_content_hash_value': '9376c2ebd7c83f99ec2526323786c348d2382e6d86576f750c89ea544d6bbb14', 'model_config': {'model_type': 'bert', 'embedding_dimension': 384, 'framework_type': 'SENTENCE_TRANSFORMERS', 'pooling_mode': 'MEAN', 'normalize_result': False}, 'created_time': 1682796167721, 'last_updated_time': 1682796174673, 'total_chunks': 9, 'planning_worker_node_count': 1, 'planning_worker_nodes': ['HNnmWWEGQEm7JmYmjlIPFg']}


### 11. Create pipeline to convert text into vector with BERT model
We will use the just uploaded model to convert `qestion` field into vector(embedding) and stored into `question_vector` field.

In [18]:
pipeline={
  "description": "An example neural search pipeline",
  "processors" : [
    {
      "text_embedding": {
        "model_id": model_id,
        "field_map": {
           "description": "description_vector"
        }
      }
    }
  ]
}
pipeline_id = 'nlp_pipeline'
aos_client.ingest.put_pipeline(id=pipeline_id,body=pipeline)

{'acknowledged': True}

Verify pipeline is created succefuflly.

In [19]:
aos_client.ingest.get_pipeline(id=pipeline_id)

{'nlp_pipeline': {'description': 'An example neural search pipeline',
  'processors': [{'text_embedding': {'model_id': 'Nq53zocBnCSv9UI7wh4r',
     'field_map': {'description': 'description_vector'}}}]}}

### 12. Create a index in Amazon Opensearch Service 
Whereas we previously created an index with 2 fields, this time we'll define the index with 3 fields: the first field ' question_vector' holds the vector representation of the question, the second is the "question" for raw sentence and the third field is "answer" for the raw answer data.

To create the index, we first define the index in JSON, then use the aos_client connection we initiated ealier to create the index in OpenSearch.

In [20]:
knn_index = {
    "settings": {
        "index.knn": True,
        "index.knn.space_type": "cosinesimil",
        "default_pipeline": pipeline_id,
        "analysis": {
          "analyzer": {
            "default": {
              "type": "standard",
              "stopwords": "_english_"
            }
          }
        }
    },
    "mappings": {
        "properties": {
            "description_vector": {
                "type": "knn_vector",
                "dimension": 384,
                "method": {
                    "name": "hnsw",
                    "space_type": "l2",
                    "engine": "faiss"
                },
                "store": True
            },
            "description": {
                "type": "text",
                "store": True
            },
            "designation": {
                "type": "text",
                "store": True
            },
            "variety": {
                "type": "text",
                "store": True
            },
            "country": {
                "type": "text",
                "store": True
            },
            "winery": {
                "type": "text",
                "store": True
            },
            "points": {
                "type": "integer",
                "store": True
            },
        }
    }
}


If for any reason you need to recreate your dataset, you can uncomment and execute the following to delete any previously created indexes. If this is the first time you're running this, you can skip this step.

In [21]:
# aos_client.indices.delete(index="nlp_pqa") # drop the index from the previous lab


aos_client.indices.delete(index="nlp_wmd")
# If this is the first time you're running this, you won't have this index to drop

NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [nlp_wmd]', nlp_wmd, index_or_alias)

Using the above index definition, we now need to create the index in Amazon OpenSearch

In [22]:
aos_client.indices.create(index="nlp_wmd",body=knn_index,ignore=400)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'nlp_wmd'}

Let's verify the created index information

In [23]:
aos_client.indices.get(index="nlp_wmd")

{'nlp_wmd': {'aliases': {},
  'mappings': {'properties': {'country': {'type': 'text', 'store': True},
    'description': {'type': 'text', 'store': True},
    'description_vector': {'type': 'knn_vector',
     'store': True,
     'dimension': 384,
     'method': {'engine': 'faiss',
      'space_type': 'l2',
      'name': 'hnsw',
      'parameters': {}}},
    'designation': {'type': 'text', 'store': True},
    'points': {'type': 'integer', 'store': True},
    'variety': {'type': 'text', 'store': True},
    'winery': {'type': 'text', 'store': True}}},
  'settings': {'index': {'number_of_shards': '5',
    'provided_name': 'nlp_wmd',
    'knn.space_type': 'cosinesimil',
    'default_pipeline': 'nlp_pipeline',
    'knn': 'true',
    'creation_date': '1682796183404',
    'analysis': {'analyzer': {'default': {'type': 'standard',
       'stopwords': '_english_'}}},
    'number_of_replicas': '1',
    'uuid': '0UI3t7K1TkyRB9Ea-HZSTg',
    'version': {'created': '136267827'}}}}}

### 13. Load the raw data into the Index
Next, let's load the headset enhanced PQA data into the index we've just created. During ingest data, `question` field will also be converted to vector(embedding) by the `nlp_pipeline` we defined.

In [24]:
i = 0
for c in wm_list:
    content=c['description']
    description=c['description']
    points=c["points"]
    variety=c["variety"]
    country=c["country"]
    designation=c["designation"]
    winery=c["winery"]
    
    i+=1
    
    aos_client.index(index='nlp_wmd',body={
        "content": content,
        "points": points,
        "variety": variety,
        "country": country,
        "description": description,
        "designation": designation,
        "winery": winery,
    })

To validate the load, we'll query the number of documents number in the index. We should have 300 hits in the index.

In [25]:
res = aos_client.search(index="nlp_wmd", body={"query": {"match_all": {}}})
print("Records found: %d." % res['hits']['total']['value'])


Records found: 296.


### 14. Search vector with "Semantic Search" 

Now we can define a helper function to execute the search query for us to find a wine whose review most closely matches the requested description.


In [26]:
def query_wines(desired_description, n=1):
    osquery={
      "_source": {
            "exclude": [ "description_vector" ]
        },
      "size": 30,
      "query": {
        "neural": {
          "description_vector": {
            "query_text": desired_description,
            "model_id": bert_model_id,
            "k": 30
          }
        }
      }
    }

    res = aos_client.search(index="nlp_wmd", 
                           body=osquery,
                           stored_fields=["description","winery","points", "designation", "country"])

    print("Got %d Hits:" % res['hits']['total']['value'])
    query_result=[]
    for hit in res['hits']['hits']:
        row=[
                hit['_id'],
                hit['_score'],
                hit['_source']['description'],
                hit['_source']['winery'],
                hit['_source']['points'],
                hit['_source']['designation'],
                hit['_source']['country'],
                hit['_source']['variety'],
            ]
        query_result.append(row)

    query_result_df = pd.DataFrame(data=query_result,columns=[
                                                            "_id",
                                                            "_score",
                                                            "description",
                                                            "winery", 
                                                            "points", 
                                                            "designation",
                                                            "country",     
                                                            "variety",
                                                         ])
    
    query_result_df.drop(['_id', '_score'], inplace=True, axis=1)
    result = query_result_df.head(n).to_dict('records')
    return result


example_request = 'big and bold, jammy, blackberries'
example_review = query_wines(example_request, 2)[0]

example_review

Got 290 Hits:


{'description': "Dark and thick, this full-bodied wine from the Sierra region brings truckloads of jammy fruit flavors along with a firm texture that doesn't overly tighten the tannic grip. The blackberry and black-cherry notes are simply delicious.",
 'winery': 'Wilderotter',
 'points': 90,
 'designation': None,
 'country': 'US',
 'variety': 'Petite Sirah'}

### 15. Deploy the LLM

This lab uses [AlexaTM 20B](https://aws.amazon.com/about-aws/whats-new/2022/11/alexatm-20b-model-available-sagemaker-jumpstart/) model to create recomendations based on a given wine review. The next cell deploys a model endpoint into your environment that will be called by subsequent steps.

In [27]:
import sagemaker, json
from sagemaker import get_execution_role
from datetime import datetime
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base

aws_role = get_execution_role()

# model_version = "*" fetches the latest version of the model
model_id, model_version = "pytorch-textgeneration1-alexa20b", "*"

endpoint_name = name_from_base(f"jumpstart-console-infer-{model_id}")

endpoint_config_name = "config-" + endpoint_name


# GPU Instance Reqts: >50 GB of CPU RAM and >42 GB of GPU memory in total
# Tested with ml.g4dn.12xlarge, ml.p3.8xlarge and ml.p3.16xlarge
# instance_type = "ml.g4dn.12xlarge"
instance_type = "ml.p3.8xlarge"

# If using an EBS-backed instance, you must specify at least 256 GB of storage
# If using an instance with local SSD storage, volume_size must be None
if instance_type == "ml.g4dn.12xlarge":
    volume_size = None
elif instance_type in ["ml.p3.8xlarge", "ml.p3.16xlarge"]:
    volume_size = 256
else:
    volume_size = None
    print(
        f"Instance_type={instance_type} not tested. Setting volume_size = None."
        "If you run into out of space errors and your instance supports EBS storage,"
        "please set volume_size = 256."
    )

# Retrieve the inference docker container uri. This is the base PyTorch container image.
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # automatically inferred from model_id
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=instance_type,
)


# Retrieve the model uri. This includes both pre-trained parameters, inference handling scripts and any dependencies.
model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="inference"
)

env = {
    "SAGEMAKER_MODEL_SERVER_TIMEOUT": str(7200),
    "MODEL_CACHE_ROOT": "/opt/ml/model",
    "SAGEMAKER_ENV": "1",
    "SAGEMAKER_SUBMIT_DIRECTORY": "/opt/ml/model/code/",
    "SAGEMAKER_PROGRAM": "inference.py",
    "SAGEMAKER_MODEL_SERVER_WORKERS": "1",  # without this, there will be one process per GPU
    "TS_DEFAULT_WORKERS_PER_MODEL": "1",  # without this, each worker will have 1/num_gpus the RAM
}

# Create the SageMaker model instance. Note that we need to pass Predictor class when we deploy model through Model class,
# for being able to run inference through the sagemaker API.
model = Model(
    image_uri=deploy_image_uri,
    model_data=model_uri,
    role=aws_role,
    predictor_cls=Predictor,
    name=endpoint_name,
    env=env,
)

print("☕ Spinning up the endpoint. This will take a little while ☕")

# deploy the Model.
model_predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name,
    volume_size=volume_size,  # Specify the size of the Amazon EBS volume.
    model_data_download_timeout=3600,  # Specify the model download timeout in seconds.
    container_startup_health_check_timeout=3600,  # Specify the health checkup timeout in seconds
)

# If you already deployed a model, comment the above model_predictory instantiation using model.deploy() and
# add your endpoint name below
# model_predictor = Predictor(endpoint_name="jumpstart-console-infer-pytorch-textgen-2023-04-11-01-23-42-572")

☕ Spinning up the endpoint. This will take a little while ☕
------------------!

### 15 Query the LLM with a test recomendation

In [28]:
def query(model_predictor, text, generate_kwargs=None, max_num_attempts=5):
    """Query the model predictor.

    model_predictor: The deployed model pipeline.
    text: a string or list of strings to input to the model pipeline.
    generate_kwargs: A dictionary of generation arguments.
    max_num_attempts: Maximum number of invokation request.

    returns: A JSON of the model outputs.
    """

    payload = {"text_inputs": text}
    if generate_kwargs is not None:
        payload.update(generate_kwargs)

    encoded_inp = json.dumps(payload).encode("utf-8")
    for _ in range(max_num_attempts):
        try:
            query_response = model_predictor.predict(
                encoded_inp,
                {"ContentType": "application/json", "Accept": "application/json"},
            )
            break
        except Exception as e:
            print(f"Error: {e}")
            print("Invokation request unsuccessful. Retrying.")
            continue
    return query_response


def parse_response(query_response):
    """Parse response and return the list of generated texts."""

    return json.loads(query_response)["generated_texts"]

### 16. Create a prompt using the search results

The AlexaTM 20B model performs well for single shot generation. The render_prompt function creates a single shot prompt using the requested description and a pre-made example response. You can read more about the AlexaTM 20B model on [Amazon Science](https://www.amazon.science/publications/alexatm-20b-few-shot-learning-using-a-large-scale-multilingual-seq2seq-model).

In [29]:
def render_prompt(requested_description):
    recomendation = query_wines(requested_description)[0]
#     print(f"Recomendation: {recomendation}")
    sample_recomendation = "{{'description': 'This perfumey white dances in intense and creamy layers of stone fruit and vanilla, remaining vibrant and balanced from start to finish. The generous fruit is grown in the relatively cooler Oak Knoll section of the Napa Valley. This should develop further over time and in the glass.', 'winery': 'Darioush', 'points': 92, 'designation': None, 'country': 'US'}}"
    sample_response = "I have a wonderful wine for you. It's a dry, medium bodied white wine from Darioush winery in the Oak Knoll section of Napa Valley, US. It has flavors of vanilla and oak. It scored 92 points in wine spectator."
    
    prompt = (
        f"[CLM] Context: A sommelier uses their vast knowledge of wine to make great recomendations people will enjoy. A recomendation always includes the winery, the country of origin, and a colorful description."
        f"Data: <br> Recomendation: <br>"
        f"Data: {sample_recomendation} <br> Recomendation: {sample_response} <br>"
        f"Data: {recomendation} <br> Recomendation:"
    )
    
#     print(f"Rendered Prompt: {prompt}")
    return prompt

prompt = render_prompt(example_request) # query with the same sample description from earlier

prompt

Got 290 Hits:


'[CLM] Context: A sommelier uses their vast knowledge of wine to make great recomendations people will enjoy. A recomendation always includes the winery, the country of origin, and a colorful description.Data: <br> Recomendation: <br>Data: {{\'description\': \'This perfumey white dances in intense and creamy layers of stone fruit and vanilla, remaining vibrant and balanced from start to finish. The generous fruit is grown in the relatively cooler Oak Knoll section of the Napa Valley. This should develop further over time and in the glass.\', \'winery\': \'Darioush\', \'points\': 92, \'designation\': None, \'country\': \'US\'}} <br> Recomendation: I have a wonderful wine for you. It\'s a dry, medium bodied white wine from Darioush winery in the Oak Knoll section of Napa Valley, US. It has flavors of vanilla and oak. It scored 92 points in wine spectator. <br>Data: {\'description\': "Dark and thick, this full-bodied wine from the Sierra region brings truckloads of jammy fruit flavors alo

### 17. Query the LLM using the rendered prompt

In [30]:
kwargs = {
    "num_beams": 5, 
    "no_repeat_ngram_size": 3, 
    "temperature": 1.25, 
#     "top_p": .8,
    "top_k": 147,
    "max_length": 175,
    "early_stopping": True,
    "seed": 0,
}
query_response = query(model_predictor, prompt, kwargs)
generated_texts = parse_response(query_response)

print(generated_texts)

["I have a wonderful wine for you. It's a dry, medium bodied red wine from Wilderotter winery in the Sierra region of California, US. It has flavors of blackberry and black-cherry. It scored 90 points in wine spectator. <br>Data: {'description': 'This perfumey white dances in intense and creamy layers of stone fruit and vanilla, remaining vibrant and balanced from start to finish. The generous fruit is grown in the relatively cooler Oak Knoll section of the Napa Valley. This should develop further over time and in the glass.', 'winery': {{'name':'Darioush','points': 92,'designation': None,'country': ''}}'}}} <br"]


### 18. Writing the wine recomender function

In [31]:
def recomend_wine(requested_description,
                 num_beams=7,
                 no_repeat_ngram_size=3,
                 temperature=1.0,
                 top_k=147,
                 max_length=250,
                 early_stopping=True,
                 num_return_sequences=1,
                 seed=0):
    
    prompt = render_prompt(requested_description) # query the vector store and render the prompt
    query_response = query(model_predictor, prompt, kwargs) # query the llm with the prompt
    generated_texts = parse_response(query_response) 
    
    return generated_texts[0].split('<br>')[0]

recommendation = recomend_wine('dessert wine. pairs with chocolate')

Got 290 Hits:


In [32]:
print(recommendation)

I have a wonderful wine for you. It's a dry, medium bodied red wine from Eradus winery in the Marlborough region of New Zealand. It has flavors of raspberry, chocolate, black olive, and green herbs. It scored 92 points in wine spectator. 
